# Advanced RAG with function calling

Advanced reasoning with function calling:
- Rewriting user's question to get a better search result.
- Multi-step function calling plan to get complex user's question.


In [2]:
import os
import json

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from openai import AzureOpenAI

MODEL = "gpt-4o"
client = AzureOpenAI(
    api_key=os.environ['AZURE_OPENAI_KEY'],
    api_version='2024-11-01-preview',
    azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
)

In [4]:
# function call
retrieve_function = {
    "name": "search_document",
    "description": "search and get Azure OpenAI and Generative AI related information.",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "Re-written and detailed question to find the right information. e.g., API, technology, and etc."
            }
        },
        "required": ["question"]
    }
}

In [5]:
from app.user_function import search_document

In [6]:
def call_tool(tool_call):
    available_functions = {
        "search_document": search_document,
    }  # only one function in this example, but you can have multiple

    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(**function_args)

    return {
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": function_name,
        "content": f"{function_response}",
    }

In [7]:
def call_llm(message_history):

    settings = {
        "model": MODEL,
        "tool_choice": "auto",
        "tools": [
            {"type": "function", "function": retrieve_function},
        ],
        "temperature": 0
    }

    response = client.chat.completions.create(
        messages=message_history, 
        **settings
    )

    message = response.choices[0].message
    message_history.append(message.to_dict())

    for tool_call in message.tool_calls or []:
        if tool_call.type == "function":
            tool_message = call_tool(tool_call)
            message_history.append(tool_message)

    return message_history

In [8]:
MAX_ITER = 5

def run_conversation(message_history, question):

    message_history.append({"role": "user", "content": question})
    
    length_of_chat = len(message_history)
    cur_iter = 0

    while cur_iter < MAX_ITER:
        message_history = call_llm(message_history)
        message = message_history[-1]
        
        cur_iter += 1
        if message['role'] == "tool":
            # run call_llm again to get the response
            continue
        else:
            answer = ""
            for past_message in message_history[length_of_chat:]:
                if past_message["role"] == "assistant" and past_message["content"] is not None:
                    answer += past_message['content'] + "\n"

            return answer, message_history

    return "exceeded max iterations", message_history

In [13]:
message_history = []

question = "What is Assistant API?"

response, message_history = run_conversation(message_history, question)
print(response)
print("-------------------")
question = "show sample code in python"
response, message_history = run_conversation(message_history, question)
print(response)

 tool: search the document of "What is Assistant API?" 
The Assistant API is a feature of the Azure OpenAI Service, currently in public preview, designed to simplify the creation of applications with advanced copilot-like experiences. It allows developers to build AI assistants that can sift through data, suggest solutions, and automate tasks. Unlike the stateless chat completions API, the Assistants API is stateful, meaning it automatically manages conversation threads and state, making it easier for developers to handle conversation context and integrate tools.

Key features of the Assistants API include:

- **Persistent Threads**: Automatically managed conversation threads that handle context window constraints.
- **Tool Integration**: Ability to access multiple tools in parallel, such as Code Interpreter and function calling.
- **Run and Run Steps**: Mechanisms to activate and track the steps an Assistant takes during a task.

The Assistants API is built on the same capabilities th

In [14]:
message_history

[{'role': 'user', 'content': 'What is Assistant API?'},
 {'content': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_eKXKhGFqxGkTEBduZMqyZ3hN',
    'function': {'arguments': '{"question":"What is Assistant API?"}',
     'name': 'search_document'},
    'type': 'function'}]},
 {'tool_call_id': 'call_eKXKhGFqxGkTEBduZMqyZ3hN',
  'role': 'tool',
  'name': 'search_document',
  'content': '# Azure OpenAI Assistants API (Preview)\n\nAssistants, a new feature of Azure OpenAI Service, is now available in public preview. Assistants API makes it easier for developers to create applications with sophisticated copilot-like experiences that can sift through data, suggest solutions, and automate tasks.\n\n## Overview\n\nPreviously, building custom AI assistants needed heavy lifting even for experienced developers. While the chat completions API is lightweight and powerful, it\'s inherently stateless, which means that developers had to manage conversation state and chat threads, tool integr